In [1]:
import numpy as np
import pandas as pd
import math
import random
import subprocess

## Шифр Хейса

In [2]:
n=4
r=6
n2=n*n

s_box=[1, 2, 3, 14, 6, 13, 11, 8, 15, 10, 12, 5, 7, 9, 0, 4]

In [3]:
class Plus_key:
    def result(self, x, key):
        return x^key

In [4]:
class S_box:
    def __init__(self, bits, s_box = None ):
        self.bits = bits
        self.s_box = s_box
    
    def result(self, x):
        if self.bits==n:
            return self.s_box[x]
        if self.bits==n2:
            s=0
            for i in range(n):
                x2=(x&15)
                s += (self.s_box[x2]<<4*i)
                x = x>>4
        return s
    
    def reverse(self, x):
        if self.bits==n:
            return self.s_box.index(x)
        if self.bits==n2:
            s=0
            for i in range(n):
                x2=(x&15)
                s += (self.s_box.index(x2)<<4*i)
                x = x>>4
        return s

In [5]:
class Shifting:
    def result(self, x):
        shifted_x=0
        for i in range(n):
            for j in range(n):
                shift = x & (1<<(4*i+j))
                if shift != 0:
                    shifted_x += 1 << (4*j+i)
        return shifted_x

In [6]:
def Heys(x, key, printing=False):
    y = x
    pk = Plus_key()
    s = S_box(n2, s_box)
    shift = Shifting()
    for i in range(r):
        y = pk.result(y, key[i])
        y = s.result(y)
        y = shift.result(y)
        if printing:
            print("round : " + str(i+1))
            print("   key = " + str(key[i]))
            print("   y = " + str(y))
    y = pk.result(y, key[r])
    if printing:
        print("round : " + str(7))
        print("   key = " + str(key[r]))
        print("   y = " + str(y))
    return y

In [7]:
key = [0 for i in range(r+1)]
for i in range(r+1):
    key[i] = random.randint(0, 2**16 - 1)
print(key)

[17223, 27685, 29356, 60662, 2813, 61893, 55656]


In [8]:
x = 1

In [9]:
Heys(x, key, True)

round : 1
   key = 17223
   y = 24305
round : 2
   key = 27685
   y = 43478
round : 3
   key = 29356
   y = 46348
round : 4
   key = 60662
   y = 56136
round : 5
   key = 2813
   y = 37707
round : 6
   key = 61893
   y = 41710
round : 7
   key = 55656
   y = 31622


31622

## Пошук диференціалів

In [10]:
def tableDF():
    s = S_box(n, s_box)
    DF=[[0 for i in range(n2)] for i in range (n2)]
    DF[0][0]=1
    for alpha in range(1,n2):
        for beta in range(1,n2):
            for x in range(n2):
                sx=s.result(x)
                sx_alpha=s.result(x^alpha)
                if sx^sx_alpha==beta:
                    DF[alpha][beta]+=1
            DF[alpha][beta]/=(n2)
    return DF

In [11]:
DF=tableDF()

In [12]:
a=["A0", "A1", "A2", "A3", "A4", "A5", "A6", "A7", "A8", "A9", "A10", "A11", "A12", "A13", "A14", "A15"]
b=["B0", "B1", "B2", "B3", "B4", "B5", "B6", "B7", "B8", "B9", "B10", "B11", "B12", "B13", "B14", "B15"]
pd.DataFrame(DF, columns = b, index = a)

,B0,B1,B2,B3,B4,B5,B6,B7,B8,B9,B10,B11,B12,B13,B14,B15
A0,1,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
A1,0,0.000,0.000,0.250,0.125,0.125,0.000,0.000,0.000,0.125,0.000,0.125,0.000,0.125,0.125,0.000
A2,0,0.000,0.125,0.125,0.000,0.125,0.000,0.125,0.000,0.000,0.000,0.000,0.125,0.250,0.000,0.125
A3,0,0.125,0.000,0.125,0.000,0.000,0.250,0.000,0.000,0.125,0.125,0.000,0.000,0.000,0.125,0.125
A4,0,0.125,0.000,0.125,0.000,0.000,0.125,0.125,0.250,0.000,0.000,0.000,0.125,0.000,0.000,0.125
A5,0,0.000,0.000,0.000,0.125,0.250,0.125,0.000,0.125,0.000,0.000,0.125,0.125,0.125,0.000,0.000
A6,0,0.000,0.000,0.125,0.000,0.125,0.000,0.000,0.000,0.000,0.250,0.125,0.125,0.000,0.125,0.125
A7,0,0.000,0.125,0.000,0.000,0.125,0.000,0.000,0.125,0.250,0.125,0.125,0.000,0.000,0.125,0.000
A8,0,0.125,0.000,0.000,0.125,0.000,0.000,0.000,0.125,0.000,0.000,0.250,0.125,0.000,0.125,0.125
A9,0,0.000,0.125,0.000,0.000,0.000,0.125,0.000,0.125,0.000,0.125,0.125,0.000,0.125,0.000,0.250


In [13]:
def create_list(alpha):
    a=[0 for i in range(n)]
    Bq=[]
    for i in range(n):
        a[n-i-1]=int(alpha%n2)
        alpha/=n2
    b1=[j for j in range(n2) if DF[a[0]][j]!=0]
    b2=[j for j in range(n2) if DF[a[1]][j]!=0]
    b3=[j for j in range(n2) if DF[a[2]][j]!=0]
    b4=[j for j in range(n2) if DF[a[3]][j]!=0]
    #print(b1)
    #print(b2)
    #print(b3)
    #print(b4)
    for j1 in range(len(b1)):
        for j2 in range(len(b2)):
            for j3 in range(len(b3)):
                for j4 in range(len(b4)):
                    b=b4[j4] + (b3[j3]<<4) + (b2[j2]<<8) + (b1[j1]<<12)
                    q=DF[a[0]][b1[j1]] * DF[a[1]][b2[j2]] * DF[a[2]][b3[j3]] * DF[a[3]][b4[j4]]
                    Bq.append([b,q])
    return Bq

In [14]:
create_list(16)

[[48, 0.25],
 [64, 0.125],
 [80, 0.125],
 [144, 0.125],
 [176, 0.125],
 [208, 0.125],
 [224, 0.125]]

In [15]:
p=0.0007

In [16]:
def DifferentialSearch2(alpha):
    shift = Shifting()
    G0=dict()
    G0[alpha]=1
    beta_dict=dict()
    for t in range(1,r):
        G1=dict()
        for g in G0:
            p_i=G0[g]
            if g not in beta_dict:
                delta_beta=create_list(g)
                beta_dict[g]=delta_beta
            else:
                delta_beta=beta_dict[g]
            #print(delta_beta)
            for j in range(len(delta_beta)):
                if delta_beta[j][0] in G1:
                    G1[delta_beta[j][0]] += p_i*delta_beta[j][1]
                else:
                    G1[delta_beta[j][0]]=p_i*delta_beta[j][1]
            #print(G1)
        G=dict()
        for g1 in G1:
            if G1[g1]>p:
                g = shift.result(g1)
                G[g]=G1[g1]
        print("round  " + str(t))
        print("G = " + str(G))
        print('---------------------------')
        G0=dict(G)
        #print()
        #print()
        #print()
    return G0

In [17]:
diff_search_result2=dict()
for i in range(1,n2):
    for j in range(4):
        alpha=i<<4*j
        G=DifferentialSearch2(alpha)
        if len(G)!=0:
            print(alpha, G)
            print()
            print("======================================")
            print()
            diff_search_result2[alpha]=G

round  1
G = {17: 0.25, 256: 0.125, 257: 0.125, 4097: 0.125, 4113: 0.125, 4353: 0.125, 4368: 0.125}
---------------------------
round  2
G = {51: 0.015625, 290: 0.0078125, 291: 0.0078125, 4131: 0.0078125, 4147: 0.0078125, 4387: 0.0078125, 4402: 0.0078125, 529: 0.0078125, 768: 0.00390625, 769: 0.00390625, 4609: 0.00390625, 4625: 0.00390625, 4865: 0.00390625, 4880: 0.00390625, 531: 0.0078125, 770: 0.00390625, 771: 0.00390625, 4611: 0.00390625, 4627: 0.00390625, 4867: 0.00390625, 4882: 0.00390625, 8211: 0.0078125, 8450: 0.00390625, 8451: 0.00390625, 12291: 0.00390625, 12307: 0.00390625, 12547: 0.00390625, 12562: 0.00390625, 8243: 0.0078125, 8482: 0.00390625, 8483: 0.00390625, 12323: 0.00390625, 12339: 0.00390625, 12579: 0.00390625, 12594: 0.00390625, 8723: 0.0078125, 8962: 0.00390625, 8963: 0.00390625, 12803: 0.00390625, 12819: 0.00390625, 13059: 0.00390625, 13074: 0.00390625, 8753: 0.0078125, 8992: 0.00390625, 8993: 0.00390625, 12833: 0.00390625, 12849: 0.00390625, 13089: 0.00390625, 131

round  3
G = {2: 0.00244140625, 34: 0.00244140625, 544: 0.00341796875, 8224: 0.000732421875, 8738: 0.0029296875, 561: 0.000885009765625, 546: 0.001953125, 8192: 0.00439453125, 8704: 0.002197265625, 12288: 0.001129150390625, 20480: 0.00103759765625, 49152: 0.00103759765625}
---------------------------
round  4
G = {136: 0.000820159912109375, 2056: 0.0008087158203125, 2184: 0.0008087158203125, 34824: 0.0013580322265625}
---------------------------
round  5
G = {}
---------------------------
round  1
G = {68: 0.25, 1024: 0.125, 1028: 0.125, 16388: 0.125, 16452: 0.125, 17412: 0.125, 17472: 0.125}
---------------------------
round  2
G = {3: 0.00390625, 19: 0.00390625, 274: 0.00390625, 275: 0.00390625, 4098: 0.0078125, 4354: 0.00390625, 4371: 0.00390625, 35: 0.00390625, 51: 0.00390625, 306: 0.00390625, 307: 0.00390625, 4130: 0.0078125, 4386: 0.00390625, 4403: 0.00390625, 545: 0.00390625, 561: 0.00390625, 816: 0.00390625, 817: 0.00390625, 4640: 0.0078125, 4896: 0.00390625, 4913: 0.00390625, 

round  3
G = {1: 0.00244140625, 17: 0.00244140625, 272: 0.00341796875, 4112: 0.000732421875, 4369: 0.0029296875, 4: 0.00244140625, 68: 0.00244140625, 1088: 0.00341796875, 16448: 0.000732421875, 17476: 0.0029296875, 204: 0.00103759765625, 17612: 0.00091552734375, 1224: 0.000946044921875, 35020: 0.00091552734375, 8: 0.00244140625, 136: 0.00244140625, 2176: 0.00341796875, 32896: 0.000732421875, 34952: 0.0029296875, 2244: 0.000946044921875, 273: 0.001953125, 4096: 0.00439453125, 4352: 0.002197265625, 1092: 0.001953125, 16384: 0.00439453125, 17408: 0.002197265625, 2184: 0.001953125, 32768: 0.00439453125, 34816: 0.002197265625, 16520: 0.000823974609375, 32836: 0.000823974609375, 49152: 0.001220703125}
---------------------------
round  4
G = {17: 0.00091552734375, 4113: 0.00091552734375, 4096: 0.00091552734375, 136: 0.001800537109375, 2048: 0.0010986328125, 32776: 0.000701904296875, 32904: 0.001800537109375, 34824: 0.000701904296875, 34944: 0.0010986328125, 8: 0.0010986328125, 2176: 0.000701

round  3
G = {32: 0.001953125, 34: 0.00244140625, 514: 0.00244140625, 546: 0.00244140625, 8704: 0.002197265625, 8706: 0.00439453125, 8738: 0.002197265625, 13059: 0.001068115234375, 21765: 0.00103759765625, 52236: 0.00103759765625, 544: 0.0009765625}
---------------------------
round  4
G = {}
---------------------------
round  5
G = {}
---------------------------
round  1
G = {64: 0.125, 68: 0.125, 1028: 0.125, 1092: 0.125, 17408: 0.125, 17412: 0.25, 17476: 0.125}
---------------------------
round  2
G = {2: 0.015625, 34: 0.015625, 544: 0.015625, 546: 0.015625, 8192: 0.03125, 8704: 0.015625, 8738: 0.015625, 3: 0.001953125, 19: 0.001953125, 274: 0.001953125, 275: 0.001953125, 4098: 0.00390625, 4354: 0.001953125, 4371: 0.001953125, 35: 0.001953125, 51: 0.001953125, 306: 0.001953125, 307: 0.001953125, 4130: 0.00390625, 4386: 0.001953125, 4403: 0.001953125, 545: 0.001953125, 561: 0.001953125, 816: 0.001953125, 817: 0.001953125, 4640: 0.00390625, 4896: 0.001953125, 4913: 0.001953125, 547: 0

round  3
G = {16: 0.001953125, 17: 0.00244140625, 257: 0.00244140625, 273: 0.00244140625, 4352: 0.002197265625, 4353: 0.00439453125, 4369: 0.002197265625, 64: 0.001953125, 68: 0.00244140625, 1028: 0.00244140625, 1092: 0.00244140625, 17408: 0.002197265625, 17412: 0.00439453125, 17476: 0.002197265625, 128: 0.001953125, 136: 0.00244140625, 2056: 0.00244140625, 2184: 0.00244140625, 34816: 0.002197265625, 34824: 0.00439453125, 34952: 0.002197265625, 204: 0.00079345703125, 17548: 0.00091552734375, 19468: 0.000732421875, 52228: 0.000701904296875, 34892: 0.00091552734375, 35852: 0.000732421875, 52232: 0.000701904296875, 52236: 0.001220703125, 52300: 0.000732421875, 52364: 0.000732421875, 272: 0.0009765625, 1088: 0.0009765625, 2176: 0.0009765625}
---------------------------
round  4
G = {34: 0.000732421875, 8226: 0.000732421875, 8192: 0.000732421875}
---------------------------
round  5
G = {}
---------------------------
round  1
G = {1: 0.125, 17: 0.125, 272: 0.25, 4097: 0.125, 4112: 0.125, 43

...

round  3
G = {16: 0.001953125, 17: 0.004150390625, 257: 0.00244140625, 273: 0.00439453125, 4352: 0.006103515625, 4353: 0.00439453125, 4369: 0.005859375, 64: 0.001953125, 68: 0.004150390625, 1028: 0.00244140625, 1092: 0.00439453125, 17408: 0.006103515625, 17412: 0.00439453125, 17476: 0.005859375, 128: 0.001953125, 136: 0.004150390625, 2056: 0.00244140625, 2184: 0.00439453125, 34816: 0.006103515625, 34824: 0.00439453125, 34952: 0.005859375, 204: 0.000701904296875, 1228: 0.000701904296875, 17544: 0.000732421875, 2252: 0.000701904296875, 3276: 0.000732421875, 34884: 0.000732421875, 52224: 0.000885009765625, 52292: 0.000823974609375, 52236: 0.0010986328125, 52360: 0.000823974609375, 52428: 0.00079345703125, 1: 0.00390625, 272: 0.00244140625, 4096: 0.006103515625, 4: 0.00390625, 1088: 0.00244140625, 16384: 0.006103515625, 8: 0.00390625, 2176: 0.00244140625, 32768: 0.006103515625, 16392: 0.0008544921875, 16520: 0.000823974609375, 18560: 0.000732421875, 18568: 0.000732421875, 32772: 0.00085449

round  4
G = {1088: 0.00079345703125, 16384: 0.00103759765625, 1: 0.0009613037109375, 17: 0.0009613037109375, 272: 0.001354217529296875, 4369: 0.00115203857421875, 8: 0.001773834228515625, 136: 0.001773834228515625, 2176: 0.00287628173828125, 32896: 0.000827789306640625, 34952: 0.002323150634765625, 273: 0.000762939453125, 4096: 0.001720428466796875, 4352: 0.00086212158203125, 2184: 0.001220703125, 32768: 0.00299072265625, 34816: 0.001495361328125}
---------------------------
round  5
G = {32904: 0.0009627342224121094}
---------------------------
14 {32904: 0.0009627342224121094}


round  1
G = {2: 0.125, 32: 0.125, 512: 0.25, 544: 0.125, 546: 0.125, 8192: 0.125, 8224: 0.125}
---------------------------
round  2
G = {16: 0.015625, 17: 0.015625, 257: 0.015625, 273: 0.015625, 4352: 0.015625, 4353: 0.03125, 4369: 0.015625, 32: 0.015625, 34: 0.015625, 514: 0.015625, 546: 0.015625, 8704: 0.015625, 8706: 0.03125, 8738: 0.015625, 64: 0.03125, 68: 0.03125, 1028: 0.03125, 1092: 0.03125, 17408: 

round  4
G = {51: 0.0007100105285644531, 4625: 0.000797271728515625, 8482: 0.000797271728515625, 12339: 0.0014944076538085938, 12594: 0.0008244514465332031, 12849: 0.0008244514465332031, 68: 0.00103759765625, 1024: 0.001617431640625, 16452: 0.002716064453125, 17472: 0.001617431640625, 4: 0.0010986328125, 16384: 0.0010986328125, 17408: 0.0010986328125, 17476: 0.0010986328125, 512: 0.000762939453125, 8226: 0.00128173828125, 8736: 0.000762939453125, 204: 0.0007143020629882812, 18500: 0.000713348388671875, 33928: 0.000736236572265625, 49356: 0.0013799667358398438, 50376: 0.0007538795471191406, 51396: 0.0007519721984863281, 17: 0.00182342529296875, 256: 0.002349853515625, 257: 0.000858306884765625, 4097: 0.000965118408203125, 4113: 0.0041351318359375, 4353: 0.000865936279296875, 4368: 0.002544403076171875, 136: 0.003269195556640625, 2048: 0.003692626953125, 2056: 0.001495361328125, 32776: 0.001773834228515625, 32904: 0.006717681884765625, 34824: 0.001499176025390625, 34944: 0.00424194335937

round  3
G = {51: 0.0045166015625, 290: 0.001953125, 291: 0.00244140625, 4131: 0.0020751953125, 4147: 0.001953125, 4387: 0.0029296875, 4402: 0.0020751953125, 529: 0.001953125, 768: 0.001220703125, 769: 0.001220703125, 4609: 0.0010986328125, 4625: 0.0010986328125, 4865: 0.001220703125, 4880: 0.0009765625, 531: 0.00244140625, 770: 0.001220703125, 771: 0.001983642578125, 4611: 0.00103759765625, 4627: 0.001251220703125, 4867: 0.002197265625, 4882: 0.001007080078125, 8211: 0.0020751953125, 8450: 0.0010986328125, 8451: 0.00103759765625, 12291: 0.0013427734375, 12307: 0.00103759765625, 12547: 0.0010986328125, 12562: 0.00115966796875, 8243: 0.001953125, 8482: 0.0010986328125, 8483: 0.001251220703125, 12323: 0.00103759765625, 12339: 0.001251220703125, 12579: 0.001220703125, 12594: 0.001007080078125, 8723: 0.0029296875, 8962: 0.001220703125, 8963: 0.002197265625, 12803: 0.0010986328125, 12819: 0.001220703125, 13059: 0.0032958984375, 13074: 0.0009765625, 8753: 0.0020751953125, 8992: 0.0009765625,

round  3
G = {17: 0.010009765625, 256: 0.00439453125, 257: 0.00634765625, 4097: 0.00439453125, 4113: 0.00439453125, 4353: 0.008544921875, 4368: 0.004150390625, 51: 0.0023193359375, 290: 0.0010986328125, 291: 0.00140380859375, 4131: 0.001129150390625, 4147: 0.001129150390625, 4387: 0.00164794921875, 4402: 0.0010986328125, 529: 0.001007080078125, 531: 0.00128173828125, 771: 0.00091552734375, 4867: 0.001129150390625, 8211: 0.00103759765625, 8243: 0.001007080078125, 12579: 0.000732421875, 8723: 0.001556396484375, 8963: 0.0010986328125, 13059: 0.001678466796875, 8753: 0.00103759765625, 102: 0.002227783203125, 580: 0.0009765625, 582: 0.001220703125, 8262: 0.001007080078125, 8294: 0.0009765625, 8774: 0.00146484375, 8804: 0.001007080078125, 1058: 0.0009765625, 1062: 0.001220703125, 1542: 0.0008544921875, 9734: 0.00103759765625, 16422: 0.001007080078125, 16486: 0.0009765625, 17446: 0.00146484375, 17926: 0.00103759765625, 26118: 0.001556396484375, 17506: 0.001007080078125, 136: 0.0205078125, 204

In [18]:
diff_search_result2

{1024: {16384: 0.000736236572265625},
 16384: {32768: 0.000732421875},
 1280: {16452: 0.000820159912109375},
 20480: {32904: 0.0007781982421875},
 2048: {16452: 0.000751495361328125},
 32768: {32904: 0.000732421875},
 3328: {16452: 0.00080108642578125},
 53248: {32904: 0.00075531005859375},
 14: {32904: 0.0009627342224121094},
 3584: {68: 0.000885009765625,
  16384: 0.000865936279296875,
  17408: 0.000759124755859375,
  17476: 0.000759124755859375,
  1024: 0.00075531005859375,
  16452: 0.001216888427734375,
  17472: 0.00075531005859375,
  4113: 0.0008974075317382812,
  32904: 0.00081634521484375},
 57344: {8: 0.0007500648498535156,
  136: 0.0009560585021972656,
  34944: 0.0008416175842285156,
  34952: 0.0008416175842285156,
  32768: 0.00084686279296875,
  34816: 0.000743865966796875,
  16452: 0.0008337497711181641,
  4113: 0.0008337497711181641,
  2048: 0.000743865966796875,
  32904: 0.001201629638671875}}

In [19]:
diff_result=[]
for alpha in diff_search_result2:
    for beta in diff_search_result2[alpha]:
        prob = diff_search_result2[alpha][beta]
        diff_result.append(((alpha, beta), prob))
sorted_diff_result=sorted(diff_result, key = lambda x: x[1], reverse = True) 

In [20]:
sorted_diff_result

[((3584, 16452), 0.001216888427734375),
 ((57344, 32904), 0.001201629638671875),
 ((14, 32904), 0.0009627342224121094),
 ((57344, 136), 0.0009560585021972656),
 ((3584, 4113), 0.0008974075317382812),
 ((3584, 68), 0.000885009765625),
 ((3584, 16384), 0.000865936279296875),
 ((57344, 32768), 0.00084686279296875),
 ((57344, 34944), 0.0008416175842285156),
 ((57344, 34952), 0.0008416175842285156),
 ((57344, 16452), 0.0008337497711181641),
 ((57344, 4113), 0.0008337497711181641),
 ((1280, 16452), 0.000820159912109375),
 ((3584, 32904), 0.00081634521484375),
 ((3328, 16452), 0.00080108642578125),
 ((20480, 32904), 0.0007781982421875),
 ((3584, 17408), 0.000759124755859375),
 ((3584, 17476), 0.000759124755859375),
 ((53248, 32904), 0.00075531005859375),
 ((3584, 1024), 0.00075531005859375),
 ((3584, 17472), 0.00075531005859375),
 ((2048, 16452), 0.000751495361328125),
 ((57344, 8), 0.0007500648498535156),
 ((57344, 34816), 0.000743865966796875),
 ((57344, 2048), 0.000743865966796875),
 ((102

## Атака (пошук ключів)

In [21]:
key

[17223, 27685, 29356, 60662, 2813, 61893, 55656]

In [22]:
def attack(prob_key, beta, messages):
    result = 0
    for (y1, y2) in messages:
        y_1 = pk.result(y1, prob_key)
        y_2 = pk.result(y2, prob_key)
        y_1 = shift.result(y_1)
        y_2 = shift.result(y_2)
        y_1 = s.reverse(y_1)
        y_2 = s.reverse(y_2)
        if y_1^y_2==beta:
            result += 1
    return (prob_key, result)

In [23]:
keys = [(i,0) for i in range(1, 2**16-1)]

iteration = 1

pk = Plus_key()
s = S_box(n2, s_box)
shift = Shifting()

for (alpha, beta), prob in sorted_diff_result:
    messages = [random.randint(1, 2**16 - 1) for i in range(5000)]
    cipher_messages = [(Heys(m, key), Heys(m^alpha, key)) for m in messages]
    
    for i in range(len(keys)):
        keys[i] = attack(keys[i][0], beta, cipher_messages)
                
    max_prob = max(keys, key = lambda x: x[1])[1]
    keys = [(i, 0) for (i,p) in keys if p == max_prob]
    
    print('Iteration', iteration)
    print('Max frequency', max_prob)
    print('Keys with max frequency', [hex(k[0]) for k in keys])
    print('---------------------------')
    
    iteration +=1
    if len(keys)==1:
        break

Iteration 1
Max frequency 8
Keys with max frequency ['0x9928', '0x992c', '0x9968', '0x996c', '0x9d28', '0x9d2c', '0x9d68', '0x9d6c', '0xd928', '0xd92c', '0xd968', '0xd96c', '0xdd28', '0xdd2c', '0xdd68', '0xdd6c']
---------------------------
Iteration 2
Max frequency 5
Keys with max frequency ['0x9928', '0x992c', '0x9968', '0x996c', '0x9d28', '0x9d2c', '0x9d68', '0x9d6c', '0xd928', '0xd92c', '0xd968', '0xd96c', '0xdd28', '0xdd2c', '0xdd68', '0xdd6c']
---------------------------
Iteration 3
Max frequency 3
Keys with max frequency ['0x9928', '0x992c', '0x9968', '0x996c', '0x9d28', '0x9d2c', '0x9d68', '0x9d6c', '0xd928', '0xd92c', '0xd968', '0xd96c', '0xdd28', '0xdd2c', '0xdd68', '0xdd6c']
---------------------------
Iteration 4
Max frequency 6
Keys with max frequency ['0x9928', '0x992c', '0x9968', '0x996c', '0x9d28', '0x9d2c', '0x9d68', '0x9d6c', '0xd928', '0xd92c', '0xd968', '0xd96c', '0xdd28', '0xdd2c', '0xdd68', '0xdd6c']
---------------------------
Iteration 5
Max frequency 6
Keys wit

In [24]:
keys

[(55656, 0)]

In [25]:
keys = [(i,0) for i in range(1, 2**16-1)]

iteration = 1

pk = Plus_key()
s = S_box(n2, s_box)
shift = Shifting()

for (alpha, beta), prob in sorted_diff_result:
    messages = [random.randint(1, 2**16 - 1) for i in range(5000)]
    
    with open(f'inputs/input_{iteration}_orig.bin', 'wb') as f:
        for text in messages:
            f.write(text.to_bytes(2, 'little'))
            
    with open(f'inputs/input_{iteration}_mut.bin', 'wb') as f:
        for text in messages:
            f.write((text^alpha).to_bytes(2, 'little'))
    
    subprocess.Popen(f'Heys.exe e 08 ./inputs/input_{iteration}_orig.bin ./outputs/output_{iteration}_orig.bin', stdin = subprocess.PIPE, shell = True).communicate(timeout = 5)
    subprocess.Popen(f'Heys.exe e 08 ./inputs/input_{iteration}_mut.bin ./outputs/output_{iteration}_mut.bin', stdin = subprocess.PIPE, shell = True).communicate(timeout = 5)

    cipher_texts = []

    with open(f'outputs/output_{iteration}_orig.bin', 'rb') as f:
        line = f.read()
        concatenated_cipher_text = int.from_bytes(line, byteorder='little')

        for i in range(len(line) // 2):
            cipher_texts.append((concatenated_cipher_text >> (i * 16)) & (2**16 - 1))
            
    mutated_cipher_texts = []

    with open(f'outputs/output_{iteration}_mut.bin', 'rb') as f:
        line = f.read()
        concatenated_cipher_text = int.from_bytes(line, byteorder='little')

        for i in range(len(line) // 2):
            mutated_cipher_texts.append((concatenated_cipher_text >> (i * 16)) & (2**16 - 1))
    
    cipher_messages = [(cipher_texts[i], mutated_cipher_texts[i]) for i in range(len(cipher_texts))]
    
    for i in range(len(keys)):
        keys[i] = attack(keys[i][0], beta, cipher_messages)
                
    max_prob = max(keys, key = lambda x: x[1])[1]
    keys = [(i, 0) for (i,p) in keys if p == max_prob]
    
    print('Iteration', iteration)
    print('Max frequency', max_prob)
    print('Keys with max frequency', [hex(k[0]) for k in keys])
    print('---------------------------')
    
    iteration +=1
    if len(keys)==1:
        break

Iteration 1
Max frequency 10
Keys with max frequency ['0xba03', '0xba07', '0xba43', '0xba47', '0xbe03', '0xbe07', '0xbe43', '0xbe47', '0xfa03', '0xfa07', '0xfa43', '0xfa47', '0xfe03', '0xfe07', '0xfe43', '0xfe47']
---------------------------
Iteration 2
Max frequency 12
Keys with max frequency ['0xba03', '0xba07', '0xba43', '0xba47', '0xbe03', '0xbe07', '0xbe43', '0xbe47', '0xfa03', '0xfa07', '0xfa43', '0xfa47', '0xfe03', '0xfe07', '0xfe43', '0xfe47']
---------------------------
Iteration 3
Max frequency 10
Keys with max frequency ['0xba03', '0xba07', '0xba43', '0xba47', '0xbe03', '0xbe07', '0xbe43', '0xbe47', '0xfa03', '0xfa07', '0xfa43', '0xfa47', '0xfe03', '0xfe07', '0xfe43', '0xfe47']
---------------------------
Iteration 4
Max frequency 6
Keys with max frequency ['0xba03', '0xba07', '0xba43', '0xba47', '0xbe03', '0xbe07', '0xbe43', '0xbe47', '0xfa03', '0xfa07', '0xfa43', '0xfa47', '0xfe03', '0xfe07', '0xfe43', '0xfe47']
---------------------------
Iteration 5
Max frequency 1
Keys 